In [1]:
x = [randn(28, 28) for i in 1:50]
y = rand(1:9, 50)

function one_hot(s)
    z = zeros(10)
    z[s] = 1
    return z
end

y = one_hot.(y)

50-element Array{Array{Float64,1},1}:
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 ⋮                                                 
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0,

In [2]:
using LinearAlgebra: dot

distance(a, b) = dot(a, a) + dot(b, b) - 2 * dot(a, b)

p1 = randn(100, 28 * 28)

p2 = randn(10, 100)


softmax(x::AbstractVector) = begin
    y = exp.(x)
    return y / sum(y)
end

loss(m1, m2) = begin
    s = 0.
    for i in 1:50
        s = s + distance(softmax(m2 * atan.(m1 * reshape(x[i], length(x[i])))), y[i])
    end
    return s
end



loss (generic function with 1 method)

In [3]:
using Zygote


grad = gradient(loss, p1, p2)
# println(grad)

([0.015539666432076544 0.019992809314227613 … -0.026304324998412905 0.02031460079841017; -0.1756432479350961 -0.9104886196233372 … 0.33491385534975904 -0.047210375760226256; … ; -0.025910891700127562 0.04745581113130033 … -0.021773553595095883 0.027104164052754835; -0.023344556536412178 0.030136720308800064 … -0.03321582682018142 0.0032871686073622434], [0.2733042330215519 -0.6292616153787068 … -0.4751746997774259 0.4805903415285258; 1.066606699787812 -0.03846525379952279 … 0.39710487056890126 0.4172380499423238; … ; 0.2737760731849428 0.7898975536577246 … 1.1073856159224005 -0.281786192486323; -0.8000552520115601 -0.5394247386482925 … -0.9803206791229057 -0.7314078474158249])

In [4]:
push!(LOAD_PATH, "../../qi/variational")
using Flux.Optimise

using VARA: parameters, set_parameters!, collect_gradients
println(length(parameters(p1, p2)))


79400


In [5]:
opt = ADAM(0.1)
x0 = parameters(p1, p2)
for i in 1:100
    grad = collect_gradients(gradient(loss, p1, p2))
    Optimise.update!(opt, x0, grad)
#     x0 -= 0.1 * grad
    set_parameters!(x0, p1, p2)
    println("loss value at epoch $i is $(loss(p1, p2)).")
end
    
    
    

loss value at epoch 1 is 76.89700584915201.
loss value at epoch 2 is 79.01717731800811.
loss value at epoch 3 is 77.32377340294182.
loss value at epoch 4 is 75.66960921007879.
loss value at epoch 5 is 74.51671090793398.
loss value at epoch 6 is 75.50714608886727.
loss value at epoch 7 is 75.20033112159115.
loss value at epoch 8 is 75.99971176287984.
loss value at epoch 9 is 75.99893816079168.
loss value at epoch 10 is 75.95389644366178.
loss value at epoch 11 is 75.9997583737614.
loss value at epoch 12 is 75.99985936115247.
loss value at epoch 13 is 75.99692599687265.
loss value at epoch 14 is 75.9805708645047.
loss value at epoch 15 is 75.54281223174377.
loss value at epoch 16 is 75.99939118945254.
loss value at epoch 17 is 75.99607173512878.
loss value at epoch 18 is 75.98461840509245.
loss value at epoch 19 is 75.63389488739598.
loss value at epoch 20 is 75.87158040126171.
loss value at epoch 21 is 75.6647079476459.
loss value at epoch 22 is 75.98071965699852.
loss value at epoch 23

In [6]:
f(x) = x^2

gradient(f, 0.1)

(0.2,)

In [8]:
using Zygote: @adjoint

@adjoint f(x) = x^2, z -> begin
    println("------------------")
    (2*x*z,)
end

gradient(f, 0.1)

------------------


(0.2,)

In [2]:
using Zygote: @adjoint
using Zygote
f1(x::Vector) = 2 .* x
f2(y::Vector) = sum(y)

g(x) = f2(f1(x))

x0 = randn(3)
gradient(g, x0)

([2.0, 2.0, 2.0],)

In [3]:
@adjoint f1(x) = f1(x), z -> begin
    println("------------------")
    ( 2 .* z ,)
end

gradient(g, x0)

------------------


([2.0, 2.0, 2.0],)